In [10]:
from selenium import webdriver
import os

In [14]:
PATH = "../../Desktop/chromedriver"
driver = webdriver.Chrome(PATH)
driver.get("https://techwithtim.net")

# Close tab: driver.close()
# Close browser: driver.quit()
# Get title of website: driver.title

<ipython-input-14-58df79691945>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [ ]:
PATH = "../../Desktop/chromedriver"
driver = webdriver.Chrome(PATH)
driver.get("https://localhost:8080/search")

def test_search(search_term):
    results = {}
    search_bar = driver.find_element_by_id("search-bar")
    search_bar.clear()
    search_bar.send_keys(search_term)
    search_results = driver.find_element_by_class("search-result")
    results["length_match"] = len(search_results) == len(find(notes_data, search_term))
    
    results["content_match"] = {}
    results["votes_match"] = {}
    results["favourites_match"] = {}
    for search_result in search_results:
        title = search_result["title"]
        note_data = notes_data[title]
        results["content_match"][title] = note_data["content"] == search_result["content"]
        results["votes_match"][title] = note_data["votes"] == search_result["votes"]
        results["favourites_match"][title] = note_data["favourites"] == search_result["favourites"]
        
    return results

In [ ]:
PATH = "../../Desktop/chromedriver"
driver = webdriver.Chrome(PATH)
driver.get("https://localhost:8080/search")

def test_tier(search_term):
    results = {}
    search_bar = driver.find_element_by_id("search-bar")
    search_bar.clear()
    search_bar.send_keys(search_term)
    search_results = driver.find_element_by_class("search-result")
    
    results["tier_match"] = {}
    for search_result in search_results:
        title = search_result["title"]
        note_data = notes_data[title]
        results["tier_match"][title] = note_data["tier"] == search_result["tier"]
        
    return results

vote_results = test_votes(accounts, note_datas)
tier_results = test_tier("data")

In [ ]:
search_terms = ["data", "js", "py", "geo"]
failed = {search_term: [] for search_term in search_terms}

for search_term in search_terms:
    results = test_search(search_term)
    if not results["length_match"]:
        failed[search_term].append("length_match")
    if not all(results["votes_match"]):
        failed[search_term].append("votes_match")
    if not all(results["favourites_match"]):
        failed[search_term].append("favourites_match")
        
failed

In [ ]:
name: dev

on:
  push:
    branches: [ dev ]
  pull_request:
    branches: [ dev ]

jobs:
  build:

    runs-on: macos-latest
    steps:
      - uses: actions/checkout@v3

      - name: Use Node.js
        uses: actions/setup-node@v3
        with:
          node-version: '12.x'

      - run: npm ci
        working-directory: ./server/

      - run: npm run build --if-present
        working-directory: ./server/
        
      - run: python tests/test-authenticate.py
        working-directory: ./server/
        
      - run: python tests/test-populate.py
        working-directory: ./server/
        
      - run: python tests/test-vote-favourite.py
        working-directory: ./server/
        
      - run: python tests/test-review.py
        working-directory: ./server/